In [2]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# for displaying variable w/o print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# <a id='0'>Content</a>

- <a href='#2'>Joining and Cancatenating</a>  
    - <a href='#21'> Database-style DataFrame Joins: Merge
    - <a href='#22'> Join: Merging on Index
    - <a href='#23'> Concatenating Along an Axis
- <a href='#3'>Reshaping Data: Pivoting</a>    
    - <a href='#32'> Pivoting "Long" to "Wide" Format
    - <a href='#33'> Pivoting "Wide" to "Long" Format

## <a id='1'> Hierarchical Indexing
aka Multi-Index (MI), access and select
- <a href='#0'> Back to TOC

Handling the index of a **single DF** before combining multiple DFs

### Indexing with external arrays

In [17]:
# MI for Series
# Unstack

se = pd.Series(np.random.randn(9),
               index=[
                         ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                         [1, 2, 3, 1, 3, 1, 2, 2, 3],
                         ['A', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B'],
                       ]
              )
se.index.names = ['letter','number','capital']
se

# imagine patients a,b,c,d; tests 1,2,3
df = se.unstack() # se -> df
df

letter  number  capital
a       1       A         -1.157719
        2       A          0.816707
        3       B          0.433610
b       1       B          1.010737
        3       B          1.824875
c       1       A         -0.997518
        2       B          0.850591
d       2       A         -0.131578
        3       B          0.912414
dtype: float64

capital               A         B
letter number                    
a      1      -1.157719       NaN
       2       0.816707       NaN
       3            NaN  0.433610
b      1            NaN  1.010737
       3            NaN  1.824875
c      1      -0.997518       NaN
       2            NaN  0.850591
d      2      -0.131578       NaN
       3            NaN  0.912414

In [18]:
# access different levels of index: levels, labels

se.index
se.index.levels[0]
se.index.labels[0]

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3], ['A', 'B']],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2], [0, 0, 1, 1, 1, 0, 1, 0, 1]],
           names=['letter', 'number', 'capital'])

Index(['a', 'b', 'c', 'd'], dtype='object', name='letter')

FrozenNDArray([0, 0, 0, 1, 1, 2, 2, 3, 3], dtype='int8')

### <a id='14'>Stacked Series vs. Unstacked DF</a>
- <a href='#0'> Back to TOC

In [49]:
# Which one is better?
# 1. se has no NA value, df has
# 2. It's easier to select multiple indice for se 
se
df

## one-level indexing
# partial indexing in the outermost level
se.loc['b']
#df.loc['b']

# indexing: inclusion
se.loc['b':'d']
se.loc[['b','c','d']]
#df.loc['b':'d']
#df.loc[['b','c','d']]

print('--------indexing in an inner level:---------')
se.loc[:,2]
se.swaplevel(0,1).loc[2] #equivalent
#df.swaplevel(0,1).loc[2]

se.loc[:,:,'A']
se.swaplevel(0,2).loc['A']
#df['A']

# access two levels
se.loc['b':'d', 2]
#df.loc[[('b',2),('c',2),('d',2)]]

se.loc[:,2,'A':'B']
se.loc['b':'d',:,'B']

# access three levels
se.loc['c':'d',2,'A':'B']
# se.loc['c':'d',2] # df can't, b/c it confuses with column, #series-power
# df.loc[[('c',2),('d',2)]]

letter  number  capital
a       1       A         -1.157719
        2       A          0.816707
        3       B          0.433610
b       1       B          1.010737
        3       B          1.824875
c       1       A         -0.997518
        2       B          0.850591
d       2       A         -0.131578
        3       B          0.912414
dtype: float64

capital               A         B
letter number                    
a      1      -1.157719       NaN
       2       0.816707       NaN
       3            NaN  0.433610
b      1            NaN  1.010737
       3            NaN  1.824875
c      1      -0.997518       NaN
       2            NaN  0.850591
d      2      -0.131578       NaN
       3            NaN  0.912414

number  capital
1       B          1.010737
3       B          1.824875
dtype: float64

letter  number  capital
b       1       B          1.010737
        3       B          1.824875
c       1       A         -0.997518
        2       B          0.850591
d       2       A         -0.131578
        3       B          0.912414
dtype: float64

letter  number  capital
b       1       B          1.010737
        3       B          1.824875
c       1       A         -0.997518
        2       B          0.850591
d       2       A         -0.131578
        3       B          0.912414
dtype: float64

...

number  letter
1       a        -1.157719
2       a         0.816707
1       c        -0.997518
2       d        -0.131578
dtype: float64

letter  number  capital
c       2       B          0.850591
d       2       A         -0.131578
dtype: float64

letter  number  capital
a       2       A          0.816707
c       2       B          0.850591
d       2       A         -0.131578
dtype: float64

letter  number  capital
b       1       B          1.010737
        3       B          1.824875
c       2       B          0.850591
d       3       B          0.912414
dtype: float64

letter  number  capital
c       2       B          0.850591
d       2       A         -0.131578
dtype: float64

In [20]:
# generate a more specific quest
x = [ (i,j,k) for i,j in [('c',1),('c',2),('d',2)] for k in ['A','B']]
se.loc[x]
# similar as above, but less control, it is a bummer to use all combos
se.loc[['c','d'],[1,2]]  # all combos 


se.loc[:,2]
df.swaplevel(0,1).loc[2]

se.loc[:,:,'A']
df['A'] # same as above but with NA

letter  number  capital
c       2       A               NaN
d       2       A         -0.131578
dtype: float64

letter  number  capital
c       1       A         -0.997518
                B               NaN
        2       A               NaN
                B          0.850591
d       2       A         -0.131578
                B               NaN
dtype: float64

letter  number  capital
c       1       A         -0.997518
        2       B          0.850591
d       2       A         -0.131578
dtype: float64

letter  number  capital
c       2       B          0.850591
d       2       A         -0.131578
dtype: float64

,capital,A,B
letter,number,,
c,2,NaN,0.850591
d,2,-0.131578,NaN


letter  capital
a       A          0.816707
c       B          0.850591
d       A         -0.131578
dtype: float64

capital,A,B
letter,,
a,0.816707,NaN
c,NaN,0.850591
d,-0.131578,NaN


letter  number
a       1        -1.157719
        2         0.816707
c       1        -0.997518
d       2        -0.131578
dtype: float64

letter  number
a       1        -1.157719
        2         0.816707
        3              NaN
b       1              NaN
        3              NaN
c       1        -0.997518
        2              NaN
d       2        -0.131578
        3              NaN
Name: A, dtype: float64

In [50]:
# MI for df

df = pd.DataFrame({'A':np.random.randn(9),'B':np.random.randn(9),'C':np.random.randn(9)},
                  index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                         [1, 2, 3, 1, 3, 1, 2, 2, 3]
                        ]
                 )
df
df.columns.names
# imagine patients a,b,c,d; tests 1,2,3; lab A,B,C

# df

df.loc['a']
df.loc['b':'d']  # the right end IS included, unlike list
df.loc[['b','d']]

df.loc[[('a',3),('b',3)]]  # tuple index
df.unstack()

A         B         C
a 1  0.188211  0.152677 -0.589488
  2  2.169461 -1.565657  1.581700
  3 -0.114928 -0.562540 -0.528735
b 1  2.003697 -0.032664  0.457002
  3  0.029610 -0.929006  0.929969
c 1  0.795253 -0.482573 -1.569271
  2  0.118110 -0.036264 -1.022487
d 2 -0.748532  1.095390 -0.402827
  3  0.584970  0.980928  0.220487

FrozenList([None])

,A,B,C
1,0.188211,0.152677,-0.589488
2,2.169461,-1.565657,1.581700
3,-0.114928,-0.562540,-0.528735


A         B         C
b 1  2.003697 -0.032664  0.457002
  3  0.029610 -0.929006  0.929969
c 1  0.795253 -0.482573 -1.569271
  2  0.118110 -0.036264 -1.022487
d 2 -0.748532  1.095390 -0.402827
  3  0.584970  0.980928  0.220487

A         B         C
b 1  2.003697 -0.032664  0.457002
  3  0.029610 -0.929006  0.929969
d 2 -0.748532  1.095390 -0.402827
  3  0.584970  0.980928  0.220487

,,A,B,C
a,3,-0.114928,-0.562540,-0.528735
b,3,0.029610,-0.929006,0.929969


A                             B                             C  \
          1         2         3         1         2         3         1   
a  0.188211  2.169461 -0.114928  0.152677 -1.565657 -0.562540 -0.589488   
b  2.003697       NaN  0.029610 -0.032664       NaN -0.929006  0.457002   
c  0.795253  0.118110       NaN -0.482573 -0.036264       NaN -1.569271   
d       NaN -0.748532  0.584970       NaN  1.095390  0.980928       NaN   

                       
          2         3  
a  1.581700 -0.528735  
b       NaN  0.929969  
c -1.022487       NaN  
d -0.402827  0.220487

In [55]:
# naming levels for row/col MI

frame.index.names   = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame
frame.swaplevel(axis=1)

state      Ohio     Colorado    
color     Green Red    Green Red
key1 key2                       
a    1        0   1        2   3
     2        4   5        6   7
b    1        8   9       10  11
     2       12  13       14  15

color     Green  Red    Green      Red
state      Ohio Ohio Colorado Colorado
key1 key2                             
a    1        0    1        2        3
     2        4    5        6        7
b    1        8    9       10       11
     2       12   13       14       15

In [52]:
# unstack inner level by default
frame.unstack()
frame.unstack(level=0) 

state  Ohio             Colorado            
color Green     Red        Green     Red    
key2      1   2   1   2        1   2   1   2
key1                                        
a         0   4   1   5        2   6   3   7
b         8  12   9  13       10  14  11  15

state  Ohio             Colorado            
color Green     Red        Green     Red    
key1      a   b   a   b        a   b   a   b
key2                                        
1         0   8   1   9        2  10   3  11
2         4  12   5  13        6  14   7  15

In [56]:
# Select MI by columns  (.loc[] for row, [] for column)

frame.head()
frame['Ohio']
frame['Ohio']['Red']
frame[ [('Ohio','Red'),('Colorado','Red')] ]

state      Ohio     Colorado    
color     Green Red    Green Red
key1 key2                       
a    1        0   1        2   3
     2        4   5        6   7
b    1        8   9       10  11
     2       12  13       14  15

color      Green  Red
key1 key2            
a    1         0    1
     2         4    5
b    1         8    9
     2        12   13

key1  key2
a     1        1
      2        5
b     1        9
      2       13
Name: Red, dtype: int64

state     Ohio Colorado
color      Red      Red
key1 key2              
a    1       1        3
     2       5        7
b    1       9       11
     2      13       15

In [57]:
# generate MI for reuse
idx_mult = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
idx_mult
pd.Series(range(3), index=idx_mult)

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

state     color
Ohio      Green    0
          Red      1
Colorado  Green    2
dtype: int64

In [59]:
%debug?

### <a id='11'> Reordering and Sorting Levels
- <a href='#0'> Back to TOC

In [14]:
frame = pd.DataFrame(np.arange(16).reshape((4, 4)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado','Colorado'],
                              ['Green', 'Red', 'Green','Red']])
frame

frame.index.names   = ['key1', 'key2']
frame.columns.names = ['state', 'color']

# swap levels based on names or axis
frame.swaplevel(0,1)
frame.swaplevel('key1', 'key2') # axis=0
frame.swaplevel(axis=1)

# swap and then sort level 0 makes more sense than sorting inner level
frame.sort_index(level=1)
frame.swaplevel(0,1).sort_index(level=0)

Ohio     Colorado    
    Green Red    Green Red
a 1     0   1        2   3
  2     4   5        6   7
b 1     8   9       10  11
  2    12  13       14  15

state      Ohio     Colorado    
color     Green Red    Green Red
key2 key1                       
1    a        0   1        2   3
2    a        4   5        6   7
1    b        8   9       10  11
2    b       12  13       14  15

state      Ohio     Colorado    
color     Green Red    Green Red
key2 key1                       
1    a        0   1        2   3
2    a        4   5        6   7
1    b        8   9       10  11
2    b       12  13       14  15

color     Green  Red    Green      Red
state      Ohio Ohio Colorado Colorado
key1 key2                             
a    1        0    1        2        3
     2        4    5        6        7
b    1        8    9       10       11
     2       12   13       14       15

state      Ohio     Colorado    
color     Green Red    Green Red
key1 key2                       
a    1        0   1        2   3
b    1        8   9       10  11
a    2        4   5        6   7
b    2       12  13       14  15

state      Ohio     Colorado    
color     Green Red    Green Red
key2 key1                       
1    a        0   1        2   3
     b        8   9       10  11
2    a        4   5        6   7
     b       12  13       14  15

### <a id='12'> Summary Statistics by Level
- <a href='#0'> Back to TOC

In [55]:
frame
frame.sum(level='key2')
frame.sum(level='color', axis=1)

state      Ohio     Colorado    
color     Green Red    Green Red
key1 key2                       
a    1        0   1        2   3
     2        4   5        6   7
b    1        8   9       10  11
     2       12  13       14  15

state  Ohio     Colorado    
color Green Red    Green Red
key2                        
1         8  10       12  14
2        16  18       20  22

color      Green  Red
key1 key2            
a    1         2    4
     2        10   12
b    1        18   20
     2        26   28

### <a id='13'> Indexing with a DataFrame's columns: columns to index
- <a href='#0'> Back to TOC

In [60]:
# set_index, reset_index

frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

frame2 = frame.set_index(['c', 'd'])
frame2
frame2.reset_index()

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## <a id='2'> Joining and Cancatenating Datasets
 <a href='#0'> Go Back To TOC</a>

### <a id='21'> Database-Style Join of DataFrames

In [3]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})


df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


#### Merge two (and only two) DFs

In [4]:
# make params explict: explicit is better than implicit
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


#### The key column has a different name in each df

In [12]:
df2 = df2.rename(columns={'key': 'key1'})

pd.merge(df1, df2, left_on='key', right_on='key1').drop(['key1'], axis=1)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


### <a id='22'> Merging on Index: ft. join
Join is a special merge when the share key is the index for all the DFs. You can join DFs or named Series. It supports joining multiple DFs all at once, unlike merge can only do two DFs at a time.
- <a href='#0'> Back to TOC

#### Join DFs

In [15]:
df1  = pd.DataFrame({'key': ['c', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
df2 = pd.DataFrame({'group_val': [3.5, 7]}, index=['b', 'a'])

df1.set_index('key').join(df2, how='outer') # set_index sorts the key/index

,value,group_val
a,2,7.0
a,3,7.0
b,1,3.5
b,4,3.5
c,0,NaN
c,5,NaN


#### Join named Series

In [16]:
se1 = pd.Series(range(3), index=list('abc'), name='s1')

# Note: no series.join
df1.set_index('key').join(se1, how='outer')

,value,s1
a,2,0
a,3,0
b,1,1
b,4,1
c,0,2
c,5,2


#### Join multiple DFs

In [17]:
df3 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
df1.set_index('key').join([pd.DataFrame(se1), df3], how='outer')

,value,s1,New York,Oregon
a,2.0,0.0,7.0,8.0
a,3.0,0.0,7.0,8.0
b,1.0,1.0,NaN,NaN
b,4.0,1.0,NaN,NaN
c,0.0,2.0,9.0,10.0
c,5.0,2.0,9.0,10.0
e,NaN,NaN,11.0,12.0
f,NaN,NaN,16.0,17.0


### <a id='23'> Concatenating Along an Axis
- <a href='#0'> Back to TOC
    
Concatenate Series, DFs, or a mixture of both. <br>
It's widely used in feature engineering during machine learning: you generate additional columns of features and horizontally stack them to the original DF. Or you have additional data coming in, you need to stack them vertically. <br>
Note: it can be tricky when you horizontally stack multiple DFs that have different indices, as shown below.

### Pandas: concatenate

In [19]:
# on axis 0 by default, more rows
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3], index=['a', 'b'])
s3 = pd.Series([5, 6], index=['a', 'g'])

pd.concat([s1, s2, s3], axis=0) # vertical
pd.concat([s1, s2, s3], axis=1) # horizontal, note the NAs created due to mismatching of the index.

a    0
b    1
a    2
b    3
a    5
g    6
dtype: int64

,0,1,2
a,0.0,2.0,5.0
b,1.0,3.0,NaN
g,NaN,NaN,6.0


#### Solution: Ignore original index of s3

In [34]:
s3.index = s2.index
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,2,5
b,1,3,6


## <a id='3'>Pivoting
 <a href='#0'> Go Back To TOC</a>

### <a id='32'> Pivoting “Long” to “Wide” Format
"Long" data form is commonly used in storing multiple time series data into a database. Pivot it into a "wide" form so that we can apply data analysis more readily.

In [34]:
long_df = pd.read_csv('examples/macrodata_cleaned_long.csv')
long_df.head()

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,unemp,5.800
2,1959-03-31,infl,0.000
3,1959-06-30,realgdp,2778.801
4,1959-06-30,unemp,5.100


In [35]:
pivoted = long_df.pivot(index='date', columns='item', values='value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


### <a id='33'> Pivoting “Wide” to “Long” Format
This time we reverse the process for data storage.
- <a href='#0'> Back to TOC

In [51]:
pd.melt(pivoted.reset_index(), id_vars='date', value_vars=['infl', 'realgdp','unemp']).head()

,date,item,value
0,1959-03-31,infl,0.00
1,1959-06-30,infl,2.34
2,1959-09-30,infl,2.74
3,1959-12-31,infl,0.27
4,1960-03-31,infl,2.31


- <a href='#0'> Back to TOC